In [2]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import itertools
import pyarrow.parquet as pq
from tqdm import tqdm

In [3]:
train = pq.read_table('train_mfti.parquet').to_pandas()

In [4]:
train

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response


### Список взаимодействий с вакансией

- 1) show_vacancy - просмотр вакансии
- 2) preview_click_vacancy - клик по карточке вакансии
- 3) **click_response - отклик со страницы вакансии**
- 4) **preview_click_response - отклик с карточки вакансии**
- 5) click_favorite - добавление вакансии в избранное  со страницы вакансии
- 6) preview_click_favorite - добавление вакансии в избраное с карточки вакансии
- 7) **click_contacts - клик на контакты со страницы вакансии**
- 8) **preview_click_contacts - клик на контакты из карточки вакансии**
- 9) **click_phone - клик на номер телефона, указанный в вакансии**
- 10) **preview_click_phone - клик на номер телефона из карточки вакансии**

Жирным выделены искомые "откликнется" и "позвонит". Меняем 1,2,5,6 - 1, остальное на 2

In [5]:
list1=['show_vacancy', 'preview_click_vacancy', 'click_favorite', 'preview_click_favorite']

list2=['click_contacts','preview_click_contacts','preview_click_phone','preview_click_response','click_phone','click_response']


In [6]:
mask1=train.iloc[:, 5:6].stack().apply(lambda x: x in list1).unstack()

mask2=train.iloc[:, 5:6].stack().apply(lambda x: x in list2).unstack()

In [7]:
mask1

,event_type
0,True
1,True
2,True
3,True
4,True
...,...
12292583,False
12292584,False
12292585,False
12292586,False


In [6]:
train.event_type[(mask1.event_type)]=1

C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_9540\2647336539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.event_type[(mask1.event_type)]=1


In [7]:
train.event_type[(mask2.event_type)]=10

C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_9540\3652181272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.event_type[(mask2.event_type)]=10


In [8]:
train=train[['user_id', 'vacancy_id_', 'event_type']]

In [9]:
train=train.drop_duplicates()

In [10]:
train_group=train.groupby(['user_id', 'vacancy_id_']).agg({'event_type':'max'})

In [62]:
train_group

event_type
user_id                          vacancy_id_            
000089d26cdd49d68839c68bc10f2cf2 177380                1
                                 181353                1
                                 195890                1
                                 205562                1
                                 211064                1
...                                                  ...
ffff62d297a347de801a1f824b4fef38 197408                1
                                 220736                1
                                 223476                1
                                 224509                1
                                 243259                1

[3287179 rows x 1 columns]

In [66]:
train_group.to_pickle('train_group.pickle')

In [2]:
train_group = pd.read_pickle('train_group.pickle')

Составим список всех пользователей:

In [55]:
train_group[:300001]

event_type
user_id                          vacancy_id_            
000089d26cdd49d68839c68bc10f2cf2 177380                1
                                 181353                1
                                 195890                1
                                 205562                1
                                 211064                1
...                                                  ...
17408f610ccc4b7e976fd14872d6e610 189469                1
                                 244659                1
                                 245061                1
                                 245485                1
                                 247755                1

[300001 rows x 1 columns]

In [77]:
train_group.index.get_level_values(0)

Index(['000089d26cdd49d68839c68bc10f2cf2', '000089d26cdd49d68839c68bc10f2cf2',
       '000089d26cdd49d68839c68bc10f2cf2', '000089d26cdd49d68839c68bc10f2cf2',
       '000089d26cdd49d68839c68bc10f2cf2', '000089d26cdd49d68839c68bc10f2cf2',
       '00008b2a006648b0b7eb409bbba203f1', '00008b2a006648b0b7eb409bbba203f1',
       '00008b2a006648b0b7eb409bbba203f1', '00008b2a006648b0b7eb409bbba203f1',
       ...
       'ffff62d297a347de801a1f824b4fef38', 'ffff62d297a347de801a1f824b4fef38',
       'ffff62d297a347de801a1f824b4fef38', 'ffff62d297a347de801a1f824b4fef38',
       'ffff62d297a347de801a1f824b4fef38', 'ffff62d297a347de801a1f824b4fef38',
       'ffff62d297a347de801a1f824b4fef38', 'ffff62d297a347de801a1f824b4fef38',
       'ffff62d297a347de801a1f824b4fef38', 'ffff62d297a347de801a1f824b4fef38'],
      dtype='object', name='user_id', length=3287179)

In [80]:
len(set(train_group.index.get_level_values(0)))

209335

In [86]:
list(set(train_group.index.get_level_values(0)))[:20000]

['45191373d232401482000db9706ec808',
 '1b6c5a89e59a400db8fb8180a9b49961',
 'ee38f5a7f58f4761bc3dabe5f2b2d2fc',
 '9618d3b8f48f4fa4b277d040fcdf574a',
 'e3ea874f98d2485faf81e8cb70da3b92',
 'e551c584e6b841a3a6b3b38f15b836fa',
 'f28e49d446e043b4b0750829cd25b8a9',
 '1fe97a905ae540e2bc5dd45a8d25d5d2',
 'a4cf50c6aaa742eebc96e617d1779394',
 '3cb3661adb7e483ba607b31674233590',
 'b705f59a505e486ca668f11b1416a93c',
 'c862e554e1b5428bb678426a097c90e7',
 '62e726666323410c9915fa5f086b3857',
 '58c766ea46334e6298220f4b07d29150',
 'fde9c69b9f204e0a93241d2e1faf858b',
 '93ce0cfd09094085b351d432e4d615b1',
 'e7ebd6e928d04bbdbf74ff8e5302e804',
 'f20f9111f8e8437d9eda330d0557730b',
 '089f453465d74de38ec5a7d9832fa642',
 '728ffa5e5b0a46108e26f1d99d508707',
 '7a6088ceac2e4c828bf04cd7a3c09856',
 '8b8bce21bf98465f8d614649f798b457',
 'a9bbdc613bc94da6b92c070c5286e835',
 '43019ddf7d3040b09be0b4f3ed72f515',
 '8bfa1a96f4fd42c4889ad2068d629e14',
 '1ff51c87f91a4652ac8a1de61e7a57ba',
 '58079eab65644b92ac0da7ebb3c62c71',
 

In [3]:
start=train_group.loc[(list(set(train_group.index.get_level_values(0)))[:20000], )].unstack()

In [8]:
sstart = start.astype(pd.SparseDtype("float", np.nan))

In [9]:
sstart

event_type                              \
vacancy_id_                          101282 127700 153586 183958 212329   
user_id                                                                   
3a705bdf693d4772939e86df86c77cb0        1.0    1.0   10.0   10.0    1.0   
3ee8b98cf01044de94ebba83f42d9246        NaN    NaN    NaN    NaN    NaN   
c7bfe7b731ed49b084587a5af92ab2b0        NaN    NaN    NaN    NaN    NaN   
e868caadf4174fb98dcff670e72b9f49        NaN    NaN    NaN    NaN    NaN   
098b738d2d3b45e788f5aaa085ec57da        NaN    NaN    NaN    NaN    NaN   
...                                     ...    ...    ...    ...    ...   
867c0823d8a04e7cbb22c8cfb34c22f5        NaN    NaN    NaN    NaN    NaN   
4bfcedca44ba49198eed4b05c23fa2b6        NaN    NaN    NaN    NaN    NaN   
643c031f72c34c588ae16b2bdd1247fd        NaN    NaN    NaN    NaN    NaN   
03b54b7499e34114afd1cecd3e859895        NaN    NaN    NaN    NaN    NaN   
a00d50ef797e4e608afd5f24c491d309        NaN    NaN    NaN    NaN    NaN   

                                                                     ...  \
vacancy_id_                      212726 213217 218851 220982 222568  ...   
user_id                                                              ...   
3a705bdf693d4772939e86df86c77cb0    1.0   10.0    1.0    1.0    1.0  ...   
3ee8b98cf01044de94ebba83f42d9246    NaN    NaN    NaN    NaN    NaN  ...   
c7bfe7b731ed49b084587a5af92ab2b0    NaN    NaN    NaN    NaN    NaN  ...   
e868caadf4174fb98dcff670e72b9f49    NaN    NaN    NaN    NaN    NaN  ...   
098b738d2d3b45e788f5aaa085ec57da    NaN    NaN    NaN    NaN    NaN  ...   
...                                 ...    ...    ...    ...    ...  ...   
867c0823d8a04e7cbb22c8cfb34c22f5    NaN    NaN    NaN    NaN    NaN  ...   
4bfcedca44ba49198eed4b05c23fa2b6    NaN    NaN    NaN    NaN    NaN  ...   
643c031f72c34c588ae16b2bdd1247fd    NaN    NaN    NaN    NaN    NaN  ...   
03b54b7499e34114afd1cecd3e859895    NaN    NaN    NaN    NaN    NaN  ...   
a00d50ef797e4e608afd5f24c491d309    NaN    NaN    NaN    NaN    NaN  ...   

                                                                            \
vacancy_id_                      115397 119731 150853 118176 122677 181078   
user_id                                                                      
3a705bdf693d4772939e86df86c77cb0    NaN    NaN    NaN    NaN    NaN    NaN   
3ee8b98cf01044de94ebba83f42d9246    NaN    NaN    NaN    NaN    NaN    NaN   
c7bfe7b731ed49b084587a5af92ab2b0    NaN    NaN    NaN    NaN    NaN    NaN   
e868caadf4174fb98dcff670e72b9f49    NaN    NaN    NaN    NaN    NaN    NaN   
098b738d2d3b45e788f5aaa085ec57da    NaN    NaN    NaN    NaN    NaN    NaN   
...                                 ...    ...    ...    ...    ...    ...   
867c0823d8a04e7cbb22c8cfb34c22f5    NaN    NaN    NaN    NaN    NaN    NaN   
4bfcedca44ba49198eed4b05c23fa2b6    1.0    1.0    1.0    NaN    NaN    NaN   
643c031f72c34c588ae16b2bdd1247fd    NaN    NaN    NaN    1.0    1.0    1.0   
03b54b7499e34114afd1cecd3e859895    NaN    NaN    NaN    NaN    NaN    NaN   
a00d50ef797e4e608afd5f24c491d309    NaN    NaN    NaN    NaN    NaN    NaN   

                                                              
vacancy_id_                      192330 193050 111562 212273  
user_id                                                       
3a705bdf693d4772939e86df86c77cb0    NaN    NaN    NaN    NaN  
3ee8b98cf01044de94ebba83f42d9246    NaN    NaN    NaN    NaN  
c7bfe7b731ed49b084587a5af92ab2b0    NaN    NaN    NaN    NaN  
e868caadf4174fb98dcff670e72b9f49    NaN    NaN    NaN    NaN  
098b738d2d3b45e788f5aaa085ec57da    NaN    NaN    NaN    NaN  
...                                 ...    ...    ...    ...  
867c0823d8a04e7cbb22c8cfb34c22f5    NaN    NaN    NaN    NaN  
4bfcedca44ba49198eed4b05c23fa2b6    NaN    NaN    NaN    NaN  
643c031f72c34c588ae16b2bdd1247fd   10.0    1.0    NaN    NaN  
03b54b7499e34114afd1cecd3e859895    NaN    NaN    NaN    NaN  
a00d5

In [10]:
BYTES_TO_MB_DIV = 0.000001
def print_memory_usage_of_data_frame(df):
    mem = round(df.memory_usage().sum() * BYTES_TO_MB_DIV, 3) 
    print("Memory usage is " + str(mem) + " MB")

In [11]:
print_memory_usage_of_data_frame(start)

Memory usage is 13104.64 MB


In [12]:
print_memory_usage_of_data_frame(sstart)

Memory usage is 3.873 MB


In [18]:
del start

In [15]:
start=train_group.loc[(list(set(train_group.index.get_level_values(0)))[20000:30000], )].unstack()

In [16]:
sstart1 = start.astype(pd.SparseDtype("float", np.nan))

In [21]:
sstart.columns = sstart.columns.droplevel()

In [23]:
sstart

vacancy_id_,101282,127700,153586,183958,212329,212726,213217,218851,220982,222568,...,115397,119731,150853,118176,122677,181078,192330,193050,111562,212273
user_id,,,,,,,,,,,,,,,,,,,,,
3a705bdf693d4772939e86df86c77cb0,1.0,1.0,10.0,10.0,1.0,1.0,10.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3ee8b98cf01044de94ebba83f42d9246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c7bfe7b731ed49b084587a5af92ab2b0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e868caadf4174fb98dcff670e72b9f49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
098b738d2d3b45e788f5aaa085ec57da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867c0823d8a04e7cbb22c8cfb34c22f5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4bfcedca44ba49198eed4b05c23fa2b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643c031f72c34c588ae16b2bdd1247fd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,1.0,10.0,1.0,NaN,NaN


In [24]:
sstart1

vacancy_id_,101296,101579,102913,104062,105276,113670,115977,120426,120763,120764,...,255827,258704,188316,222001,122984,128639,167223,237634,242648,242942
user_id,,,,,,,,,,,,,,,,,,,,,
11fb99445087455c8a9be6a80be77ff9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3709df34505142848bb374b40e2c394c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a55ed5e9f73c47f3a2f2a09a9bed6f6c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229fd95f340d44f7ad7ffba275b72658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aad6540bb52a44e1baf1e737022dd214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4ca97457516457dafee16fc0c5eeab8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2e1a87e1d57e4d32bdb79a572f71b984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2821857c797847ddbaf5676d53625d95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df=pd.concat([sstart,sstart1], sort=False, axis=0)

In [38]:
print_memory_usage_of_data_frame(df)

Memory usage is 41.121 MB


In [29]:
df.to_pickle('df_group.pickle')

In [31]:
a=20000
for i in range(17):
    a+=10000

    start=train_group.loc[(list(set(train_group.index.get_level_values(0)))[a:a+10000], )].unstack()
    sstart = start.astype(pd.SparseDtype("float", np.nan))
    sstart.columns = sstart.columns.droplevel()
    
    df=pd.concat([df,sstart], sort=False, axis=0)

In [32]:
a

190000

In [33]:
df.to_pickle('df_groupa_a.pickle')

In [35]:
start=train_group.loc[(list(set(train_group.index.get_level_values(0)))[200000:], )].unstack()
sstart = start.astype(pd.SparseDtype("float", np.nan))
sstart.columns = sstart.columns.droplevel()
    
df=pd.concat([df,sstart], sort=False, axis=0)

In [36]:
df.to_pickle('df_groupa_a.pickle')

In [39]:
df

vacancy_id_,101282,127700,153586,183958,212329,212726,213217,218851,220982,222568,...,250060,164836,238508,239209,253690,133817,152169,190060,173189,181861
user_id,,,,,,,,,,,,,,,,,,,,,
3a705bdf693d4772939e86df86c77cb0,1.0,1.0,10.0,10.0,1.0,1.0,10.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3ee8b98cf01044de94ebba83f42d9246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c7bfe7b731ed49b084587a5af92ab2b0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e868caadf4174fb98dcff670e72b9f49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
098b738d2d3b45e788f5aaa085ec57da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887f6df4d37a477ca7d27c1c49cc53e4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cc66c230c6c642b6aaebd833b6bfbadb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9afc8b217a194208bc9085d4816d8b10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df.dtypes

vacancy_id_
101282    Sparse[float64, nan]
127700    Sparse[float64, nan]
153586    Sparse[float64, nan]
183958    Sparse[float64, nan]
212329    Sparse[float64, nan]
                  ...         
133817    Sparse[float64, nan]
152169    Sparse[float64, nan]
190060    Sparse[float64, nan]
173189    Sparse[float64, nan]
181861    Sparse[float64, nan]
Length: 155705, dtype: object

In [45]:
df = df.fillna(0)

In [47]:
df.to_pickle('df_groupa_a.pickle')

Подготовка тестового датасета

In [31]:
test_public_mfti = pq.read_table('test_public_mfti.parquet').to_pandas()
train = pq.read_table('train_mfti.parquet').to_pandas()

#Забираем записи только пользователей которые представлены тест датасете
mask=train.iloc[:, 3:4].stack().apply(lambda x: x in list(test_public_mfti.cookie_id)).unstack()
test_r=train[(mask.cookie_id)]

list1=['show_vacancy', 'preview_click_vacancy', 'click_favorite', 'preview_click_favorite']
list2=['click_contacts','preview_click_contacts','preview_click_phone','preview_click_response','click_phone','click_response']

mask1=test_r.iloc[:, 5:6].stack().apply(lambda x: x in list1).unstack()
mask2=test_r.iloc[:, 5:6].stack().apply(lambda x: x in list2).unstack()

test_r.event_type[(mask1.event_type)]=1
test_r.event_type[(mask2.event_type)]=10

test_r=test_r[['vacancy_id_', 'cookie_id', 'event_type']]


test_res=test_r.groupby(['cookie_id', 'vacancy_id_']).agg({'event_type':'max'}).unstack().fillna (0)
test_res.columns = test_res.columns.droplevel(0)

C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_8360\2652971155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_r.event_type[(mask1.event_type)]=1
C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_8360\2652971155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_r.event_type[(mask1.event_type)]=1
C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_8360\2652971155.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_

In [33]:
test_res.to_pickle('test_result_1,10.pickle')

In [3]:
test_res=pd.read_pickle('test_result_1,10.pickle')
df=pd.read_pickle('df_groupa_a.pickle')

In [9]:
test_public_mfti = pq.read_table('test_public_mfti.parquet').to_pandas()

Алгоритм

In [14]:
sum_metric_5=[]
sum_metric_10=[]
for random_user in tqdm(test_res.index):
    #try:
    #Находим пересечение вакансий нашего датасета и вакансий пользователя
    
    vacancy_watched =list(set(test_res.loc[random_user, :][test_res.loc[random_user, :]>0].index)& set(df.columns))
    

    #Найдем 30 наиболее похожих пользователей:

    df_random_user=df[vacancy_watched]


    df_top_10=df_random_user.T.sum()

    # df_top_30.sort_values(ascending=False)[0:10] - первые 10 пользователей включая и самого

    data_same_user=df_random_user.loc[list(df_top_10.sort_values(ascending=False)[0:10].index), :]
   
    #добавляем в начало самого пользователя(если он есть у нас все равно мы будем занулять все вакансии )
        
    data_same_user=pd.concat([test_res.loc[(test_res.index==random_user)][vacancy_watched], data_same_user])

    # строим матрицу схожести 
    
    user_similarity = pairwise_distances( data_same_user, metric='cosine').round(2)

    # Отбираем топ 10 похожих пользователей

    s=pd.Series(index=data_same_user.index[1:], data=(1-user_similarity[0][1:]))
    
      

    #Перемножаем на получившиеся коэф схожести ряды вакансий наших пользлвателей и складываем их
    
    kf=s.sort_values(ascending=False)[0:10]

    itogo=df.loc[kf.index, :]

    d=np. zeros(155705)

    for i in kf.index:
        d+=itogo.loc[i, :].to_numpy()*kf[i]
    
    last_sort=pd.Series(index=itogo.columns, data=d)

        # Зануляем вакансии которые у нас уже использовались

    last_sort[vacancy_watched]=0

    a_5=set(last_sort.sort_values(ascending=False)[:5].index)
    a_10=set(last_sort.sort_values(ascending=False)[:10].index)
    b=set(test_public_mfti.vacancy_id_[test_public_mfti.cookie_id==random_user].to_list()[0])
    
    sum_metric_5.append(len(a_5 & b))
    sum_metric_10.append(len(a_10 & b))
    #except:
     #  print(random_user)

100%|██████████| 772/772 [8:20:16<00:00, 38.88s/it]  


In [22]:
print('Среднее количество вакансий правильно предсказанных для рекомендаций из 5 вакансий: ', sum(sum_metric_5)/len(sum_metric_5))
print('Точность:', 100*sum(sum_metric_5)/(5*len(sum_metric_5)), '%')

Среднее количество вакансий правильно предсказанных для рекомендаций из 5 вакансий:  0.2603626943005181
Точность: 5.2072538860103625 %


In [24]:
print('Среднее количество вакансий правильно предсказанных для рекомендаций из 10 вакансий: ', sum(sum_metric_10)/len(sum_metric_10))
print('Точность:', 100*sum(sum_metric_10)/(10*len(sum_metric_10)), '%')

Среднее количество вакансий правильно предсказанных для рекомендаций из 10 вакансий:  0.42487046632124353
Точность: 4.248704663212435 %
